In [1]:
import pandas as pd
import yaml
from sqlalchemy import create_engine, inspect, text

In [5]:


with open('../config_fill.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_co = config['CO_SA']
    config_etl = config['ETL_PRO']

# Construct the database URL
url_co = (f"{config_co['drivername']}://{config_co['user']}:{config_co['password']}@{config_co['host']}:"
          f"{config_co['port']}/{config_co['dbname']}")
url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@{config_etl['host']}:"
           f"{config_etl['port']}/{config_etl['dbname']}")
# Create the SQLAlchemy Engine
co_oltp = create_engine(url_co)
etl_conn = create_engine(url_etl)

inspector2=inspect(co_oltp)
inspector = inspect(etl_conn)
tnames = inspector.get_table_names()
tnames2=inspector2.get_table_names(schema='sales')
print(tnames2)
if not tnames:
    with etl_conn.connect() as conn:  
        with open('../sqlscripts.yml', 'r') as f:
            sql = yaml.safe_load(f)
            for key, val in sql.items():
                conn.execute(text(val))
        conn.commit() 


['sales_person_quota_history', 'country_region_currency', 'sales_reason', 'credit_card', 'sales_tax_rate', 'currency', 'sales_territory', 'currency_rate', 'sales_territory_history', 'customer', 'shopping_cart_item', 'person_credit_card', 'special_offer', 'special_offer_product', 'sales_order_header_sales_reason', 'sales_order_detail', 'sales_order_header', 'sales_person', 'store']


## Extract

In [6]:
dim_sales_territory=pd.read_sql_table('sales_territory',co_oltp,schema='sales')

dim_sales_territory

,territory_id,name,country_region_code,group_name,sales_ytd,sales_last_year,cost_ytd,cost_last_year,rowguid,modified_date
0,1,Northwest,US,North America,7.887187e+06,3.298694e+06,0.0,0.0,43689A10-E30B-497F-B0DE-11DE20267FF7,2008-04-30 00:00:00+00:00
1,2,Northeast,US,North America,2.402177e+06,3.607149e+06,0.0,0.0,00FB7309-96CC-49E2-8363-0A1BA72486F2,2008-04-30 00:00:00+00:00
2,3,Central,US,North America,3.072175e+06,3.205014e+06,0.0,0.0,DF6E7FD8-1A8D-468C-B103-ED8ADDB452C1,2008-04-30 00:00:00+00:00
3,4,Southwest,US,North America,1.051085e+07,5.366576e+06,0.0,0.0,DC3E9EA0-7950-4431-9428-99DBCBC33865,2008-04-30 00:00:00+00:00
4,5,Southeast,US,North America,2.538667e+06,3.925071e+06,0.0,0.0,6DC4165A-5E4C-42D2-809D-4344E0AC75E7,2008-04-30 00:00:00+00:00
5,6,Canada,CA,North America,6.771829e+06,5.693989e+06,0.0,0.0,06B4AF8A-1639-476E-9266-110461D66B00,2008-04-30 00:00:00+00:00
6,7,France,FR,Europe,4.772398e+06,2.396540e+06,0.0,0.0,BF806804-9B4C-4B07-9D19-706F2E689552,2008-04-30 00:00:00+00:00
7,8,Germany,DE,Europe,3.805202e+06,1.307950e+06,0.0,0.0,6D2450DB-8159-414F-A917-E73EE91C38A9,2008-04-30 00:00:00+00:00
8,9,Australia,AU,Pacific,5.977815e+06,2.278549e+06,0.0,0.0,602E612E-DFE9-41D9-B894-27E489747885,2008-04-30 00:00:00+00:00
9,10,United Kingdom,GB,Europe,5.012905e+06,1.635823e+06,0.0,0.0,05FC7E1F-2DEA-414E-9ECD-09D150516FB5,2008-04-30 00:00:00+00:00


## Transform
